# Convolutional Neural Networks for Ground Objects_Classification

## Project: Write an Algorithm for Ground Objects Classification

---

In this notebook, convolutional neural networks will be used to build an algorithm for ground objects classification based on drone orthomaic images. There are 5 different classes: CropA, CropB, Tree, Hay, and Building. In data folder, Dr. Jili Li offered images, with size: 375 x 375.

---
### Why We're Here 

Currently, in this real-world, the accuracy gournd objects classification based on drone orthomaic images is around 70% to 80%. 


### The Road Ahead

Here, break the notebook into separate steps.

* [Step 1](#step1): Import Datasets
* [Step 2](#step2): Create a Model to Classify objects
* [Step 3](#step3): Train the Model
* [Step 4](#step4): Test the Model

---
<a id='step1'></a>
## Step 1: Import Datasets

All images are in .tif format.

In [1]:
import numpy as np
from glob import glob

# load filenames
image_files = np.array(glob("data/*/*/*.tif"))
# print number of images in dataset
print('There are %d total images.' % len(image_files))

There are 284 total images.


In [2]:
import os
from torchvision import datasets
import torchvision.transforms as transforms

data_dir = 'data/'
train_dir = os.path.join(data_dir, 'train/')
valid_dir = os.path.join(data_dir, 'valid/')
test_dir = os.path.join(data_dir, 'test/')

# data transform for training
data_transform_train = transforms.Compose([transforms.RandomRotation(30),
                                           transforms.RandomHorizontalFlip(),
                                           transforms.RandomResizedCrop(224),
                                           transforms.ToTensor(),
                                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# data transform for validation and tests
data_transform = transforms.Compose([transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_data = datasets.ImageFolder(train_dir, transform=data_transform_train)
valid_data = datasets.ImageFolder(valid_dir, transform=data_transform)
test_data = datasets.ImageFolder(test_dir, transform=data_transform)

In [3]:
import torch

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 30

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size,
                                          num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)
loaders_scratch = {'train': train_loader, 'valid': valid_loader,
                  'test': test_loader}

<a id='step2'></a>
## Step 2: Create a Model to Classify objects

Here, use Convolutional Neural Network to build model.
It includes three Convolutional layers, each of them followed by a pooling layer.
After convolutional layers, there are two fully connected hidden layers.
Drop out layers are also adopted in this model

In [4]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    ### create an architecture
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        # input size = 224 * 224 * 3 image tensor
        self.conv_1 = nn.Conv2d(3, 8, 3, padding=1)
        # 112 * 112 * 8
        self.conv_2 = nn.Conv2d(8, 16, 3, padding=1)
        # 56 * 56 * 16
        self.conv_3 = nn.Conv2d(16, 32, 3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(32*28*28, 500)
        self.fc2 = nn.Linear(500, 133)
        
        self.dropout = nn.Dropout(0.25)
    
    def forward(self, x):
        ## Define forward behavior
        x = self.pool(F.relu(self.conv_1(x)))
        x = self.pool(F.relu(self.conv_2(x)))
        x = self.pool(F.relu(self.conv_3(x)))
        
        x = x.view(-1, 32*28*28)
        x = self.dropout(x)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
use_cuda = torch.cuda.is_available()
if use_cuda:
    model_scratch.cuda()

<a id='step3'></a>
## Step 3: Train the Model

Before train the model, first set criterion and optimizer.

In [5]:
import torch.optim as optim

### set loss function
criterion_scratch = nn.CrossEntropyLoss()

### set optimizer
optimizer_scratch = optim.SGD(model_scratch.parameters(), lr=0.005)

In [6]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            ## record the average training loss
            train_loss += (1/(batch_idx + 1)) * (loss.item() - train_loss)
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
            
        valid_loss = valid_loss/len(loaders['valid'].sampler)

            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss    
    # return trained model
    return model


In [7]:
# train the model
model_scratch = train(200, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'model_scratch.pt')



Epoch: 1 	Training Loss: 4.862064 	Validation Loss: 4.797994
Validation loss decreased (inf --> 4.797994).  Saving model ...
Epoch: 2 	Training Loss: 4.759767 	Validation Loss: 4.675023
Validation loss decreased (4.797994 --> 4.675023).  Saving model ...
Epoch: 3 	Training Loss: 4.613300 	Validation Loss: 4.456644
Validation loss decreased (4.675023 --> 4.456644).  Saving model ...
Epoch: 4 	Training Loss: 4.300485 	Validation Loss: 3.914404
Validation loss decreased (4.456644 --> 3.914404).  Saving model ...
Epoch: 5 	Training Loss: 3.472538 	Validation Loss: 2.531382
Validation loss decreased (3.914404 --> 2.531382).  Saving model ...
Epoch: 6 	Training Loss: 2.205625 	Validation Loss: 1.920668
Validation loss decreased (2.531382 --> 1.920668).  Saving model ...
Epoch: 7 	Training Loss: 1.949517 	Validation Loss: 1.806087
Validation loss decreased (1.920668 --> 1.806087).  Saving model ...
Epoch: 8 	Training Loss: 1.789902 	Validation Loss: 1.763120
Validation loss decreased (1.80608

Epoch: 89 	Training Loss: 0.343710 	Validation Loss: 0.568770
Epoch: 90 	Training Loss: 0.388944 	Validation Loss: 0.551098
Epoch: 91 	Training Loss: 0.361023 	Validation Loss: 0.608087
Epoch: 92 	Training Loss: 0.399821 	Validation Loss: 0.554086
Epoch: 93 	Training Loss: 0.348459 	Validation Loss: 0.524965
Validation loss decreased (0.550578 --> 0.524965).  Saving model ...
Epoch: 94 	Training Loss: 0.367505 	Validation Loss: 0.526616
Epoch: 95 	Training Loss: 0.339849 	Validation Loss: 0.564500
Epoch: 96 	Training Loss: 0.368201 	Validation Loss: 0.524181
Validation loss decreased (0.524965 --> 0.524181).  Saving model ...
Epoch: 97 	Training Loss: 0.460010 	Validation Loss: 0.558834
Epoch: 98 	Training Loss: 0.312997 	Validation Loss: 0.558305
Epoch: 99 	Training Loss: 0.388929 	Validation Loss: 0.645981
Epoch: 100 	Training Loss: 0.376316 	Validation Loss: 0.549251
Epoch: 101 	Training Loss: 0.340745 	Validation Loss: 0.577354
Epoch: 102 	Training Loss: 0.392984 	Validation Loss: 

<a id='step4'></a>
## Step 4: Test the Model


In [9]:
# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

<All keys matched successfully>

In [10]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [11]:
# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

Test Loss: 0.163932


Test Accuracy: 92% (52/56)
